In [17]:
from power_perceiver.data_loader import HRVSatellite
from power_perceiver.transforms.satellite import PatchSatellite
from pathlib import Path
import einops
import numpy as np

In [6]:
hrv = HRVSatellite(
    data_path=Path("/home/jack/dev/ocf/power_perceiver/data_for_testing"),
    #transforms=[PatchSatellite()],
    )

In [8]:
hrv_batch = hrv[0]

In [10]:
patched = PatchSatellite()(hrv_batch)
patched

<xarray.Dataset>
Dimensions:          (example: 32, channel: 1, time: 31, y: 16, x: 16, patch: 16)
Coordinates:
    channel_name     (example, channel) object 'HRV' 'HRV' 'HRV' ... 'HRV' 'HRV'
    time_utc         (example, time) datetime64[ns] 2020-09-12T09:05:00 ... 2...
    x_geostationary  (example, x) float32 -8.446e+05 -8.406e+05 ... -6.296e+05
    x_osgb           (example, y, x) float32 2.017e+05 2.062e+05 ... 4.742e+05
    y_geostationary  (example, y) float32 4.819e+06 4.823e+06 ... 4.748e+06
    y_osgb           (example, y, x) float32 6.417e+05 6.412e+05 ... 4.533e+05
  * patch            (patch) MultiIndex
  - y_patch          (patch) int64 0 0 0 0 1 1 1 1 2 2 2 2 3 3 3 3
  - x_patch          (patch) int64 0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3
Dimensions without coordinates: example, channel, time, y, x
Data variables:
    data             (example, time, channel, y, x, patch) int16 260 260 ... 138

In [28]:
einops_patched = einops.rearrange(
    hrv_batch["data"].values,
    "example time channel (y y_patch) (x x_patch) -> example time channel y x (y_patch x_patch)",
    x_patch=4,
    y_patch=4,
    )

einops_patched.shape

(32, 31, 1, 16, 16, 16)

In [29]:
patched["data"].shape

(32, 31, 1, 16, 16, 16)

In [30]:
np.testing.assert_array_almost_equal(
    patched["data"],
    einops_patched,
)

In [31]:
# Try "un-patching"
np.testing.assert_array_almost_equal(
    hrv_batch["data"],
    einops.rearrange(
        einops_patched,
        "example time channel y x (y_patch x_patch) -> example time channel (y y_patch) (x x_patch)",
        x_patch=4,
        y_patch=4,
        ),
)